This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import collections
import requests
# Importing bs4 for tentative use
from bs4 import BeautifulSoup

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = ('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&order=asc&api_key='+API_KEY)
res = requests.get(url)
json_data = res.json()
# An alternative way to check out the data:
html_data = res.text
soup = BeautifulSoup(html_data)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(type(json_data))
print(json_data.keys())
print(json_data['dataset'].keys())
json_data['dataset']['data'][0]

<class 'dict'>
dict_keys(['dataset'])
dict_keys(['id', 'dataset_code', 'database_code', 'name', 'description', 'refreshed_at', 'newest_available_date', 'oldest_available_date', 'column_names', 'frequency', 'type', 'premium', 'limit', 'transform', 'column_index', 'start_date', 'end_date', 'data', 'collapse', 'order', 'database_id'])


['2017-01-02',
 34.99,
 35.94,
 34.99,
 35.8,
 None,
 44700.0,
 1590561.0,
 None,
 None,
 None]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
cols = json_data['dataset']['column_names']
print(cols)
data = json_data['dataset']['data']
print(data[0:5])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
[['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None], ['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-04', 35.48, 35.51, 34.75, 35.19, None, 54408.0, 1906810.0, None, None, None], ['2017-01-05', 35.02, 35.2, 34.73, 35.06, None, 48412.0, 1692326.0, None, None, None], ['2017-01-06', 34.91, 35.21, 34.91, 35.04, None, 27507.0, 964046.0, None, None, None]]


In [6]:
opens = []
for i in data:
    if type(i[1]) == float:
        opens.append(i[1])
max_open = max(opens)
min_open = min(opens)
print('Opening Prices 2017 -- High: '+str(max_open)+' Low: '+str(min_open))

Opening Prices 2017 -- High: 53.11 Low: 34.0


In [7]:
change = []
for i in data:
    if (type(i[2]) == float) and (type(i[3])==float):
        change.append((i[2]-i[3]))
max_change = max(change)
print('Largest change in one day based on High and Low price: '+str(max_change))

Largest change in one day based on High and Low price: 2.8100000000000023


In [8]:
two_day_change = []
for row in range(0,len(data)):
    try:
        if (type(data[row][4])==float) and (type(data[row+1][4])==float):
            two_day_change.append(data[row+1][4]-data[row][4])
    except IndexError:
        continue
max_two_day_change = max(two_day_change)
print('Largest change between any two days during 2017: '+str(max_two_day_change))

Largest change between any two days during 2017: 1.7199999999999989


In [9]:
trading_volumes = []
for i in data:
    if type(i[6])==float:
        trading_volumes.append(i[6])
trading_volume_avg = sum(trading_volumes)/len(trading_volumes)
print('Average daily traded volume for 2017: '+str(trading_volume_avg))

Average daily traded volume for 2017: 89124.33725490196


In [10]:
from statistics import median
trading_volume_median = median(trading_volumes)
print('Median daily traded volume for 2017: '+str(trading_volume_median))

Median daily traded volume for 2017: 76286.0
